# Hybrid Quantum Convolutional Neural Netwoks for Earth Observation Classification
![](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/4609443/9656571/9647979/sebas9-3134785-large.gif)


Please refer to the following articles to get more insight about this topic:
    - Sebastianelli, A., Zaidenberg, D. A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021). On circuit-based hybrid quantum neural networks for remote sensing imagery classification. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 15, 565-580.
    - Zaidenberg, D. A., Sebastianelli, A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021, July). Advantages and bottlenecks of quantum machine learning for remote sensing. In 2021 IEEE International Geoscience and Remote Sensing Symposium IGARSS (pp. 5680-5683). IEEE.
    - Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
    - Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.
    - https://qiskit.org/documentation/machine-learning/tutorials/index.html
    - https://pennylane.ai/qml/demos_qml.html


# Download the dataset

In this study, we address the challenge of land use and land cover classification using Sentinel-2 satellite images. The Sentinel-2 satellite images are openly and freely accessible provided in the Earth observation program Copernicus. We present a novel dataset based on Sentinel-2 satellite images covering 13 spectral bands and consisting out of 10 classes with in total 27,000 labeled and geo-referenced images. We provide benchmarks for this novel dataset with its spectral bands using state-of-the-art deep Convolutional Neural Network (CNNs). With the proposed novel dataset, we achieved an overall classification accuracy of 98.57%. The resulting classification system opens a gate towards a number of Earth observation applications. We demonstrate how this classification system can be used for detecting land use and land cover changes and how it can assist in improving geographical maps.

Get more information [here](https://github.com/phelber/EuroSAT).

![](https://github.com/phelber/EuroSAT/blob/master/eurosat_overview_small.jpg?raw=true)




For this tutorial we are using the EuroSAT RGB dataset, accessible via wget. The following cell takes care of downloading, unzipping and preparing the dataset.

In [ ]:
!rm -r /content/EuroSAT.zip
!rm -r /content/EuroSAT
!wget https://madm.dfki.de/files/sentinel/EuroSAT.zip --no-check-certificate
!unzip -q /content/EuroSAT.zip
!mv 2750 EuroSAT
!ls -l /content/EuroSAT

## Cloning repository with base code
Basic functions are wrapped in seprated files to keep the notebook clean. Feel free to explore the code.

In [ ]:
import sys
!rm -r /content/QML-tutorial
!git clone https://github.com/alessandrosebastianelli/QML-tutorial.git
if '/content/QML-tutorial' not in sys.path: sys.path.append('/content/QML-tutorial')

## Install missing packages

In [ ]:
!pip install --upgrade hqm --no-deps -q

# Network train and validation

In [8]:
# Suppressing warning
import warnings
warnings.filterwarnings('ignore')

def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
    fxn()

from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning)

import numpy as np
np.seterr(all="ignore")

from utils.DatasetHandler import DatasetHandler
from models.PyTorchModel import PyTorchModel
# Configuration file, please read it carefully
from config import *

import pennylane as qml
import torch.optim as optim
import torch.nn as nn
import torch
import os


# Allows to reload modified code without restarting the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from hqm.classification.hcnn import HybridLeNet5
from hqm.circuits.angleencoding import BasicEntangledCircuit
from hqm.layers.basiclayer import BasicLayer

Load the dataset

In [6]:
################################ Initialize Dataset Handler ################################
################################ and print classes          ################################
print('Loading Dataset')
dh = DatasetHandler(DATASET_ROOT)

classes = []
for i, c in enumerate(dh.classes):
    cl = c.split(os.path.sep)[-1]
    classes.append(cl)
classes.sort()
print('[*] Classes: {}'.format(classes))

################################ Load image paths and labels ################################
imgs, labels = dh.load_paths_labels(DATASET_ROOT, classes=classes)
print('[*] Size: {}'.format(len(imgs)))

################################# Training-Validation Split #################################
tra_imgs, tra_lbls, val_imgs, val_lbls = dh.train_validation_split(imgs, labels, SPLIT_FACTOR)
print('[*] Training Size:   {}'.format(len(tra_imgs)))
print('[*] Validation Size: {}'.format(len(val_imgs)))

Loading Dataset
[*] Classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
[*] Size: 27000
[*] Training Size:   21600
[*] Validation Size: 5400


Initialize hybrid model

In [9]:
dev = qml.device("lightning.qubit", wires=NUM_QUBITS)
qcircuit = BasicEntangledCircuit(n_qubits=NUM_QUBITS, n_layers=NUM_LAYERS, dev=dev)
qlayer = BasicLayer(qcircuit, aiframework='torch')

In [10]:
print('Initialize Quantum Hybrid Neural Network')
network = HybridLeNet5(qlayer=qlayer, in_shape=(3,64,64), ou_dim=10)

optimizer = optim.SGD(network.parameters(), lr=LEARNING_RATE, momentum = MOMENTUM)
criterion = nn.CrossEntropyLoss()

print('Printing Quantum Circuit Parameters')
print('[*] Number of Qubits:   {}'.format(NUM_QUBITS))
print('[*] Number of R Layers: {}'.format(NUM_LAYERS))
print('[*] Number of Shots:    {}'.format(NUM_SHOTS))

# This class wrap a PyTorch model. It is only needed to mask basic function, like model training.
model = PyTorchModel(network, criterion, optimizer)

Initialize Quantum Hybrid Neural Network
Printing Quantum Circuit Parameters
[*] Number of Qubits:   4
[*] Number of R Layers: 2
[*] Number of Shots:    1


Fit the model

In [11]:
tra_set = [tra_imgs, tra_lbls]
val_set = [val_imgs, val_lbls]
model.fit(EPOCHS, tra_set, val_set, classes, batch_size=BATCH_SIZE, es=None, tra_size = None, val_size = None)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
model.curves()